# Проект по теме "Машинное обучение для текстов"
## 1. Описание проекта

**Проект для «Викишоп»**

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, другими словами, клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. Необходимо построить модель детектирования негативных сообщений для дальнейшей обработки сервисной службой.

**Цели**:
Обучить модель классифицировать комментарии на позитивные и негативные. В распоряжении набор данных с разметкой о токсичности сообщений. Построить модель со значением метрики качества F1 не меньше 0.75.

**Инструкция по выполнению проекта**:
- Загрузить и подготовить данные.
- Обучить разные модели.
- Сделать выводы.

### 1.1. Описание данных
Данные находятся в файле **toxic_comments.csv.**

**text** - текст комментария,
**toxic** - целевой признак.

**План выполнения работы:**

- Подготовка Данных
- Обучение моделей
    - 2.1 Logistic Regression
    - 2.2 NB-SVM
    - 2.3 Linear SVC
- Выводы

### 1.2. Подготовка данных
Подключаем библиотеки:
- **pandas** - для работы с таблицами
- **sklearn** - инструменты машинного обучения (модели классификации, метрики для исследования качества моделей, разделение данных, предобработка данных)
- **nltk** - для лемматизации и фильтрации стоп-слов

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.utils import shuffle
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.dummy import DummyClassifier

import nltk
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer as WNL
from nltk.tokenize import word_tokenize
from nltk import pos_tag

import re
import spacy
import torch
import transformers
from tqdm import notebook
from tqdm.notebook import tqdm
from wordcloud import WordCloud
from time import time

import warnings
warnings.filterwarnings("ignore")

In [3]:
try:
    toxic_data = pd.read_csv('toxic.csv')
except:
    toxic_data = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv')
    
toxic_data.sample(5)

,Unnamed: 0,text,toxic
127667,127799,stfu ==\n\nWHY DONT U STFU AND GET A LIFE! IF ...,1
84420,84501,I don't know whether the map per se is correct...,0
129212,129345,"Two as in disambiguation? Farmbrough, .",0
142503,142656,UNBLOCK - the question of integrity is not an ...,0
102047,102144,Please remember to sign your posts by adding f...,0


In [4]:
toxic_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [5]:
toxic_data = toxic_data.drop('Unnamed: 0', axis=1)

In [6]:
toxic_data.shape

(159292, 2)

In [7]:
toxic_data['text'] = toxic_data['text'].str.lower()

In [8]:
toxic_data.duplicated().sum()

45

In [9]:
toxic_data = toxic_data.drop_duplicates()

In [10]:
toxic_data.isna().sum()

text     0
toxic    0
dtype: int64

In [11]:
toxic_data['toxic'].value_counts()
print(f"Процент объектов класса 1 к общему объёму датасета: {(sum(toxic_data['toxic']) / len(toxic_data) * 100):.2f}%")

Процент объектов класса 1 к общему объёму датасета: 10.15%


###   1.3. Лемматизация с помощью WordNetLemmatizer

- Удалим пунктуацию и лишние пробелы
- Удалим стоп-слова 
- Проведём лемматизацию слов с помощью WordNetLemmatizer() из библиотеки nltk

In [15]:
def get_wordnet_pos(word):    
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [16]:
lemmatizer = WNL()
nltk.download('averaged_perceptron_tagger')
sentence = "The striped bats are hanging on their feet for best"
print([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(sentence)])

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


['The', 'strip', 'bat', 'be', 'hang', 'on', 'their', 'foot', 'for', 'best']


In [17]:
def lemm_wnl(text):
    text = re.sub(r'[^a-zA-z ]', ' ', text)
    tokens = nltk.word_tokenize(text)
    text = ' '.join([lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in tokens])
    return text

In [18]:
lemm_wnl(sentence)

'The strip bat be hang on their foot for best'

In [19]:
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

def lemm_spacy(text):
    text = re.sub(r'[^a-zA-z ]', ' ', text)
    doc = nlp(text)
    text = ' '.join([token.lemma_ for token in doc])
    return text

lemm_spacy(sentence)

'the stripe bat be hang on their foot for good'

In [20]:
tqdm.pandas()
toxic_data['lemm_text_WML'] = toxic_data['text'].progress_apply(lemm_wnl)

  0%|          | 0/159247 [00:00<?, ?it/s]

In [21]:
toxic_data.sample(5)

,text,toxic,lemm_text_WML
44289,"good intentions \n\nhi yan,\n\ni fail to see t...",0,good intention hi yan i fail to see the good i...
44949,"""\n\n why are you removing the only remaining ...",0,why be you remove the only remain prose look i...
102408,"""auto|1=65.242.111.254|2=autoblocked because y...",0,auto autoblocked because your ip address be re...
36375,"it makes me feel better, and i like that. i ne...",0,it make me feel well and i like that i never t...
97115,you don't and yet while you ask that others o ...,0,you don t and yet while you ask that others o ...


In [22]:
toxic_data['lemm_text_spacy'] = toxic_data['text'].progress_apply(lemm_spacy)

  0%|          | 0/159247 [00:00<?, ?it/s]

In [23]:
toxic_data.sample(5)

,text,toxic,lemm_text_WML,lemm_text_spacy
151726,this article needs a rewrite. \n\ni did some c...,0,this article need a rewrite i do some cleaning...,this article need a rewrite I do some clea...
259,y r we doing this: pat,0,y r we do this pat,y r we do this pat
74011,south house \n\ni was sure he was in south hou...,0,south house i be sure he be in south house in ...,south house I be sure he be in south house ...
72852,defamatory articles \n\nyou are behind the def...,0,defamatory article you be behind the defamator...,defamatory article you be behind the defama...
36701,do i know you? ==because you are a fggt!\n do ...,1,do i know you because you be a fggt do i know ...,do I know you because you be a fggt do ...


In [24]:
#toxic_data = toxic_data.drop('Unnamed: 0', axis=1)
toxic_data.rename(columns={"lemm_text_WML": "lemm_text_WNL"}, inplace=True)

features_wnl = toxic_data['lemm_text_WNL']
features_spacy = toxic_data['lemm_text_spacy']
target = toxic_data['toxic']

In [25]:
#Разделим данные на тренировочную и тестовую выборки
RANDOM_STATE, CV = 1234, 5
features_train_wnl, features_test_wnl,\
features_train_spacy, features_test_spacy,\
target_train, target_test = train_test_split(features_wnl, features_spacy, target,
                                             test_size=0.1, stratify=target, random_state=RANDOM_STATE)

## 2. Обучение через Pipelines

Найдём метрику f1 для константной модели **Dummy**. Будем предсказывать все твиты нетоксичными ('toxic'=0)

### 2.1. DummyClassifier

In [26]:
model_dm = DummyClassifier()
model_dm.get_params().keys()

dict_keys(['constant', 'random_state', 'strategy'])

In [27]:
stop_words = set(stopwords.words('english')) 

pipeline_dc = Pipeline(
    [
        ("vectorizer", TfidfVectorizer(stop_words=list(stop_words))),
        ("model_dm", DummyClassifier(random_state=RANDOM_STATE)),
    ]
)

parameter_grid_dc = dict(
    #vectorizer__ngram_range = [(1, 1), (1, 2)],
    model_dm__strategy = ['most_frequent', 'uniform'],
)

pipeline_dc

Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(stop_words=['don', "it's", 'have', 'on',
                                             'each', 'some', 'yourself', 'any',
                                             'the', "mustn't", 'down', 'hers',
                                             'shan', 't', 'wouldn', 'is',
                                             "mightn't", "hasn't", 'does',
                                             'wasn', 'at', 'why', 'where',
                                             "aren't", 'because', 'be', 'both',
                                             'can', "don't", 'yours', ...])),
                ('model_dm', DummyClassifier(random_state=1234))])

In [28]:
def RSCV(pipeline, grid, features_train):
    
    rscv = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=grid,
    scoring='f1',
    cv=CV,
    random_state=RANDOM_STATE,
    n_jobs=-1,
    verbose=1,
    error_score='raise')
    
    t0 = time()
    rscv.fit(features_train, target_train)
    print(f"Done in {time() - t0:.3f}s")
    print(f"Best parameters combination found:")
    best_parameters = rscv.best_estimator_.get_params()
    
    for param_name in sorted(grid.keys()):
        print(f"{param_name}: {best_parameters[param_name]}")
    print(f"Best f1: {rscv.best_score_:.3f}")
    
    return rscv

In [29]:
dc_rscv_wnl = RSCV(pipeline_dc, parameter_grid_dc, features_train_wnl)

Fitting 5 folds for each of 2 candidates, totalling 10 fits
Done in 70.884s
Best parameters combination found:
model_dm__strategy: uniform
Best f1: 0.168


In [30]:
dc_rscv_spacy = RSCV(pipeline_dc, parameter_grid_dc, features_train_spacy)

Fitting 5 folds for each of 2 candidates, totalling 10 fits
Done in 75.545s
Best parameters combination found:
model_dm__strategy: uniform
Best f1: 0.168


### 2.2. Logistic Regression

In [31]:
model_logr = LogisticRegression()
model_logr.get_params().keys()

dict_keys(['C', 'class_weight', 'dual', 'fit_intercept', 'intercept_scaling', 'l1_ratio', 'max_iter', 'multi_class', 'n_jobs', 'penalty', 'random_state', 'solver', 'tol', 'verbose', 'warm_start'])

Обучение, подбор гиперпараметров логистической регрессии и кросс-валидацию проведём с помощью GridSearchCV, подбирать будем гиперпараметры **С и max_iter**

In [32]:
pipeline_logr = Pipeline(
    [
        ("vectorizer", TfidfVectorizer(stop_words=list(stop_words))),
        ("model_logr", LogisticRegression(random_state=RANDOM_STATE, class_weight='balanced')),
    ]
)

parameter_grid_logr = dict(
    #vectorizer__ngram_range = [(1, 1), (1, 2)],
    model_logr__C = [10, 20],
    model_logr__solver = ['liblinear', 'lbfgs']
)

In [33]:
logr_rscv_wnl = RSCV(pipeline_logr, parameter_grid_logr, features_train_wnl)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Done in 950.148s
Best parameters combination found:
model_logr__C: 10
model_logr__solver: liblinear
Best f1: 0.756


In [34]:
logr_rscv_spacy = RSCV(pipeline_logr, parameter_grid_logr, features_train_spacy)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Done in 1060.104s
Best parameters combination found:
model_logr__C: 10
model_logr__solver: lbfgs
Best f1: 0.763


### 2.3. SGDClassifier

In [35]:
model_sgdc = SGDClassifier()
model_sgdc.get_params().keys()

dict_keys(['alpha', 'average', 'class_weight', 'early_stopping', 'epsilon', 'eta0', 'fit_intercept', 'l1_ratio', 'learning_rate', 'loss', 'max_iter', 'n_iter_no_change', 'n_jobs', 'penalty', 'power_t', 'random_state', 'shuffle', 'tol', 'validation_fraction', 'verbose', 'warm_start'])

In [36]:
pipeline_sgdc = Pipeline(
    [
        ("vectorizer", TfidfVectorizer(stop_words=list(stop_words))),
        ("model_sgdc", SGDClassifier(random_state=RANDOM_STATE, class_weight='balanced', 
                                     learning_rate='optimal', penalty='l2')),
    ]
)

parameter_grid_sgdc = dict(
    #vectorizer__ngram_range = [(1, 1), (1, 2)],
    model_sgdc__loss = ['modified_huber', 'perceptron'],
    model_sgdc__alpha = [0.1, 0.2],
)

In [37]:
sgdc_rscv_wnl = RSCV(pipeline_sgdc, parameter_grid_sgdc, features_train_wnl)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Done in 521.313s
Best parameters combination found:
model_sgdc__alpha: 0.2
model_sgdc__loss: perceptron
Best f1: 0.690


In [38]:
sgdc_rscv_spacy = RSCV(pipeline_sgdc, parameter_grid_sgdc, features_train_spacy)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Done in 984.403s
Best parameters combination found:
model_sgdc__alpha: 0.2
model_sgdc__loss: perceptron
Best f1: 0.701


Данная модель быстрее Логистической регрессии в три раза, но значение f1 меньше на 0.015, но всё ещё выше ТЗ.

### 2.4. MultinomialNB

In [39]:
from sklearn.naive_bayes import MultinomialNB
model_mnb = MultinomialNB()
model_mnb.get_params().keys()

dict_keys(['alpha', 'class_prior', 'fit_prior'])

In [40]:
pipeline_mnb = Pipeline(
    [
        ("vectorizer", TfidfVectorizer(stop_words=list(stop_words))),
        ("model_mnb", MultinomialNB()),
    ]
)

parameter_grid_mnb = dict(
    #vectorizer__ngram_range = [(1, 1), (1, 2)],
    model_mnb__alpha = [0.1, 0.2, 0.5],
)

In [41]:
mnb_rscv_wnl = RSCV(pipeline_mnb, parameter_grid_mnb, features_train_wnl)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Done in 748.005s
Best parameters combination found:
model_mnb__alpha: 0.1
Best f1: 0.629


In [42]:
mnb_rscv_spacy = RSCV(pipeline_mnb, parameter_grid_mnb, features_train_spacy)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Done in 815.903s
Best parameters combination found:
model_mnb__alpha: 0.1
Best f1: 0.641


### 2.5. Лучшая модель на тестовой выборке

In [43]:
logr_predict = logr_rscv_spacy.predict(features_test_spacy)
f1_lr = f1_score(target_test, logr_predict)
print(f"Показатель f1 на тестовой выборке: {f1_lr:.3f}")
accuracy_lr = accuracy_score(target_test, logr_predict)
print(f"Accuracy на логистической регрессии {accuracy_lr:.3f}")

Показатель f1 на тестовой выборке: 0.780
Accuracy на логистической регрессии 0.952


- Лемматизация spacy точнее и быстрее.
- Показатель f1=0.78 на тестовой выборке удовлетворяет условию задачи.
- Accuracy на тестовой выборке у логистической регрессии составляет 0.952

## 3. Выводы

- Данные о токсичности твитов успешно загружены и обработаны:
- Лемматизация проведена с помощью **WordNetLemmatizer** библиотеки nltk и **Spacy**
- Знаки пунктуации, а также лишние пробелы удалены
- Стоп слова удалены в процессе векторизации(список взят из библиотеки nltk)
- Корпус обработран с помощью pipelin-ов, содержащих **TfidfVectorizer** и одну из трёх наиболее распространённых моделей классификации текста: **LogisticRegression, SCDG, MultinomialNB**
- f1 моделей в диапазоне 0.63-0.76. Максимальный показатель f1 на тестовой выборке получен для **LogisticRegression**: 0.78

## 4. BERT на выборке (для тренировки)

### 4.1. Подготовка данных и увеличение скорости

In [44]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

from tensorflow.python.client import device_lib
device_lib.list_local_devices()
tf.test.is_built_with_cuda()
tf.debugging.set_log_device_placement(True)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


2023-08-08 17:07:17.778161: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [45]:
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

The dtype policy mixed_float16 may run slowly because this machine does not have a GPU. Only Nvidia GPUs with compute capability of at least 7.0 run quickly with mixed_float16.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once


In [46]:
toxic_data_5000 = toxic_data.sample(5000).reset_index(drop=True)
print(f"Процент объектов класса 1 к общему объёму датасета: {(sum(toxic_data_5000['toxic'])/50):.2f}%")

Процент объектов класса 1 к общему объёму датасета: 9.84%


### 4.2. BERT-токенизация

In [47]:
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-cased')

tokenized = toxic_data_5000['text'].apply(
    lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=64, truncation=True))

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])
attention_mask = np.where(padded != 0, 1, 0)

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

### 4.3. Загрузка DistillBERT модели и создание эмбедингов

In [48]:
model_bert = transformers.DistilBertModel.from_pretrained('distilbert-base-cased')

Downloading:   0%|          | 0.00/411 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/251M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
%%time
batch_size = 5
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
    batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
    attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
    with torch.no_grad():
        batch_embeddings = model_bert(batch, attention_mask=attention_mask_batch)
        
    embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())
    del batch, attention_mask_batch, batch_embeddings
    torch.cuda.empty_cache()

  0%|          | 0/1000 [00:00<?, ?it/s]

### 4.4 Испытание лучшей модели на эмбедингах

In [ ]:
features_train = np.concatenate(embeddings)
target_train = toxic_data_5000['toxic']
features_train.shape

In [ ]:
pipeline_logr = Pipeline(
    [
        #("vectorizer", TfidfVectorizer(stop_words=list(stop_words))),
        ("model_logr", LogisticRegression(random_state=RANDOM_STATE, class_weight='balanced')),
    ]
)

parameter_grid_logr = dict(
    #vectorizer__ngram_range = [(1, 1), (1, 2)],
    model_logr__C = [10, 20],
    model_logr__solver = ['liblinear', 'lbfgs']
)

In [ ]:
%%time
logr_rscv = RSCV(pipeline_logr, parameter_grid_logr, features_train)

f1 = 0.652

## 5. Итоговый вывод
**В ходе работы над проектом было сделано:**
- Подготовлены данные для обучения на моделях.
- Данные разделены на обучающую и тестовою выборку в соотношении 9:1.
- Обучены модели через пайплайн с векторизацией и выбраны лучшие из них валидацией.
- Исследованы параметры качества моделей.

- Исходные данные обладают большим количеством признаков. Так как TF-IDF превращают текст в численные значения, лучшими моделями стали LogisticRegression и SGDClassifier.

- На тестовой выбоке по метрике F1 лучше всего себя показал LogisticRegression.

- Для небольшой (в 5000 ед.) выборки была проведена токенизацация и векторизация на DistillBERT-ом имеющимися мощностями. Преобразованные данные были классифицированы LogisticRegression. Значения f1 и accuracy получились незначительными.